In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import mlflow

In [3]:
# set the dagshub tracking server

mlflow.set_tracking_uri("https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow")

In [4]:
import dagshub
dagshub.init(repo_owner='aniketnandanwar09', repo_name='uber-demand-prediction', mlflow=True)

Accessing as aniketnandanwar09

Initialized MLflow to track repo "aniketnandanwar09/uber-demand-prediction"

Repository aniketnandanwar09/uber-demand-prediction initialized!

In [5]:
# load the training and test data

train_data_path = "../data/processed/train.csv"
test_data_path = "../data/processed/test.csv"

train_df = pd.read_csv(train_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

test_df = pd.read_csv(test_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [6]:
# missing value in training data

train_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [7]:
# missing values in the test data

test_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [8]:
# make X_train and y_train

X_train = train_df.drop(columns=["total_pickups"])

y_train = train_df["total_pickups"]

In [9]:
X_train.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185.0,4


In [10]:
# make X_test and y_test

X_test = test_df.drop(columns=["total_pickups"])

y_test = test_df["total_pickups"]

In [11]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,39.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,37.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,41.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,38.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,35.0,1


In [12]:
from sklearn import set_config

set_config(transform_output="pandas")

In [13]:
# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)

In [14]:
encoder

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('ohe',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['region', 'day_of_week'])])

In [15]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

In [16]:
import optuna
import tqdm 

d:\uber-demand-prediction\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# set the experiment

mlflow.set_experiment("Model Selection")

2025/07/03 12:50:11 INFO mlflow.tracking.fluent: Experiment with name 'Model Selection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/12a2c2f8972b4d9389ac2636e4442a0d', creation_time=1751527209678, experiment_id='0', last_update_time=1751527209678, lifecycle_stage='active', name='Model Selection', tags={}>

In [18]:
def objective(trial):
    # start the child run
    with mlflow.start_run(nested=True) as child:
        
        # model name search space
        list_of_models = ["LR", "RF", "GBR", "XGBR"]
        model_name = trial.suggest_categorical("model_name", list_of_models)
    
        if model_name == "LR":
            model = LinearRegression()
    
        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,100,step=10)
            max_depth_rf = trial.suggest_int("max_depth_rf",3,10)
            model = RandomForestRegressor(n_estimators=n_estimators_rf, 
                                          max_depth=max_depth_rf, 
                                          random_state=42, n_jobs=-1)
    
        elif model_name == "GBR":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,100,step=10)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",1e-4,1e-1, log=True)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb, 
                                              learning_rate=learning_rate_gb,
                                             random_state=42)
    
        elif model_name == "XGBR":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,100,step=10)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",1e-4,1e-1, log=True)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",3,10)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                learning_rate=learning_rate_xgb,
                                max_depth=max_depth_xgb)
    
        # log the model name
        mlflow.log_param("model_name",model_name)
        
        # log the model parameters
        mlflow.log_params(model.get_params())
        
        # fit on the data
        model.fit(X_train_encoded,y_train)
    
        # get the predictions
        y_pred = model.predict(X_test_encoded)
    
        # calculate the loss
        loss = mean_absolute_percentage_error(y_test, y_pred)
    
        # log the metric
        mlflow.log_metric("MAPE",loss)
        return loss

In [19]:
# optimize the objective function

with mlflow.start_run(run_name="best_model", nested=True) as parent:

    # create a study object
    study = optuna.create_study(study_name="model_selection", direction="minimize")
    # optimize the objective function
    study.optimize(func=objective, n_trials=50, n_jobs=-1)
    
    # log the best parameters
    mlflow.log_params(study.best_params)
    # log the best error value
    mlflow.log_metric("Best_MAPE", study.best_value)

[I 2025-07-03 12:50:34,671] A new study created in memory with name: model_selection


🏃 View run judicious-conch-93 at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0/runs/2e44327aa4044a62a5e8c4b6f30a6e61
🧪 View experiment at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0
🏃 View run selective-sow-784 at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0/runs/983633b254fb49298aaddb582de03235
🧪 View experiment at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0
🏃 View run dapper-moth-794 at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0/runs/f6a5095cb3d0407b832d3f54aff85e91
🧪 View experiment at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0
🏃 View run incongruous-mare-435 at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0/runs/c696a969fbe64acc8b2796e65d0acede
🧪 View experiment at: https://dagshub.com/aniketnandanwar09/

[I 2025-07-03 12:51:17,645] Trial 5 finished with value: 0.07934790285463077 and parameters: {'model_name': 'LR'}. Best is trial 5 with value: 0.07934790285463077.
[I 2025-07-03 12:51:20,622] Trial 10 finished with value: 0.07934790285463077 and parameters: {'model_name': 'LR'}. Best is trial 5 with value: 0.07934790285463077.


🏃 View run redolent-hen-154 at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0/runs/579d65724d8b440997dfab901dfeb2d0
🧪 View experiment at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-07-03 12:51:23,611] Trial 9 finished with value: 0.07934790285463077 and parameters: {'model_name': 'LR'}. Best is trial 5 with value: 0.07934790285463077.


...

🏃 View run gentle-hound-814 at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0/runs/fe2d48eb07d7496f923618ba1cef8de8
🧪 View experiment at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0
🏃 View run sincere-yak-891 at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0/runs/a1f06d188b9c4362a4ede7600748adca
🧪 View experiment at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-07-03 12:55:18,045] Trial 37 finished with value: 3.315160649722866 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 10, 'learning_rate_gb': 0.0718465890129626}. Best is trial 5 with value: 0.07934790285463077.
[I 2025-07-03 12:55:19,075] Trial 43 finished with value: 3.611504077911377 and parameters: {'model_name': 'XGBR', 'n_estimators_xgb': 10, 'learning_rate_xgb': 0.059143547957673064, 'max_depth_xgb': 10}. Best is trial 5 with value: 0.07934790285463077.


🏃 View run grandiose-crow-854 at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0/runs/2fd9be7d9e684941be7e4d9319731bc7
🧪 View experiment at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-07-03 12:55:29,156] Trial 36 finished with value: 0.17712952157922202 and parameters: {'model_name': 'RF', 'n_estimators_rf': 10, 'max_depth_rf': 7}. Best is trial 5 with value: 0.07934790285463077.


🏃 View run best_model at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0/runs/97f0507dbd3244f899754f1234588bae
🧪 View experiment at: https://dagshub.com/aniketnandanwar09/uber-demand-prediction.mlflow/#/experiments/0


In [20]:
# best value

study.best_value


0.07934790285463077

In [21]:
# best parameters
study.best_params

{'model_name': 'LR'}

In [22]:
# model value counts

study.trials_dataframe()['params_model_name'].value_counts()

params_model_name
LR      27
XGBR     9
GBR      8
RF       6
Name: count, dtype: int64

In [23]:
from optuna.visualization import (
    plot_optimization_history, 
    plot_parallel_coordinate, 
    plot_param_importances
)

In [24]:
plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
              'y': [6.442261622531529, 0.07934790285463077, 3.4535969440357497,
                    1.4621895698187166, 0.12902721652852625, 0.07934790285463077,
                    0.19833877206585285, 0.346799273930311, 6.518127366962824,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    6.468470573425293, 0.12869255703306878, 0.07934790285463077,
                    0.07934790285463077, 6.210500248145075, 6.090576171875,
                    6.523750305175781, 0.3468547109487106, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.17712952157922202, 3.315160649722866, 0.07934790285463077,
                    0.07934790285463077, 2.9876532554626465, 2.8572630882263184,
                    2.420549154281616, 3.611504077911377, 2.7592480182647705,
                    2.8949763774871826, 0.07934790285463077, 0.07934790285463077,
                    3.9665421426490477, 3.7694027255269673]},
             {'mode': 'lines',
              'name': 'Best Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
              'y': [6.442261622531529, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077, 0.07934790285463077,
                    0.07934790285463077, 0.07934790285463077]},
             {'marker': {'color': '#cccccc'},
              'mode': 'markers',
              'name': 'Infeasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Optimization History Plot'},
               'xaxis': {'title': {'text': 'Trial'}},
               'yaxis': {'title': {'text': 'Objective Value'}}}
})

In [25]:
plot_parallel_coordinate(study, params=["model_name"])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'dimensions': [{'label': 'Objective Value',
                              'range': [0.07934790285463077, 6.523750305175781],
                              'values': [6.442261622531529, 0.07934790285463077,
                                         3.4535969440357497, 1.4621895698187166,
                                         0.12902721652852625, 0.07934790285463077,
                                         0.19833877206585285, 0.346799273930311,
                                         6.518127366962824, 0.07934790285463077,
                                         0.07934790285463077, 0.07934790285463077,
                                         6.468470573425293, 0.12869255703306878,
                                         0.07934790285463077, 0.07934790285463077,
                                         6.210500248145075, 6.090576171875,
                                         6.523750305175781, 0.3468547109487106,
                                         0.07934790285463077, 0.07934790285463077,
                                         0.07934790285463077, 0.07934790285463077,
                                         0.07934790285463077, 0.07934790285463077,
                                         0.07934790285463077, 0.07934790285463077,
                                         0.07934790285463077, 0.07934790285463077,
                                         0.07934790285463077, 0.07934790285463077,
                                         0.07934790285463077, 0.07934790285463077,
                                         0.07934790285463077, 0.07934790285463077,
                                         0.17712952157922202, 3.315160649722866,
                                         0.07934790285463077, 0.07934790285463077,
                                         2.9876532554626465, 2.8572630882263184,
                                         2.420549154281616, 3.611504077911377,
                                         2.7592480182647705, 2.8949763774871826,
                                         0.07934790285463077, 0.07934790285463077,
                                         3.9665421426490477, 3.7694027255269673]},
                             {'label': 'model_name',
                              'range': [0, 3],
                              'ticktext': [GBR, LR, RF, XGBR],
                              'tickvals': [0, 1, 2, 3],
                              'values': [0, 1, 0, 0, 2, 1, 2, 2, 0, 1, 1, 1, 3, 2,
                                         1, 1, 0, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1,
                                         1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 3, 3,
                                         3, 3, 3, 3, 1, 1, 0, 0]}],
              'labelangle': 30,
              'labelside': 'bottom',
              'line': {'color': [6.442261622531529, 0.07934790285463077,
                                 3.4535969440357497, 1.4621895698187166,
                                 0.12902721652852625, 0.07934790285463077,
                                 0.19833877206585285, 0.346799273930311,
                                 6.518127366962824, 0.07934790285463077,
                                 0.07934790285463077, 0.07934790285463077,
                                 6.468470573425293, 0.12869255703306878,
                                 0.07934790285463077, 0.07934790285463077,
                                 6.210500248145075, 6.090576171875,
                                 6.523750305175781, 0.3468547109487106,
                                 0.07934790285463077, 0.07934790285463077,
                                 0.07934790285463077, 0.07934790285463077,
                                 0.07934790285463077, 0.07934790285463077,
                                 0.07934790285463077, 0.07934790285463077,
                                 0.07934790285463077, 0.07934790285463077,
                                 0.07934790285463077, 0.07934790285463077,
                      

In [26]:
def tune_ridge(trial):
    # hyperparameter space
    alpha = trial.suggest_float("alpha",30,100)
    
    # make the model object
    ridge = Ridge(alpha=alpha, random_state=42)
    
    # train the model
    ridge.fit(X_train_encoded, y_train)
    
    # get predictions
    y_pred = ridge.predict(X_test_encoded)
    
    # calculate loss
    loss = mean_absolute_percentage_error(y_test, y_pred)

    return loss

In [27]:
# create study

study = optuna.create_study(study_name="tune_model", direction="minimize")

[I 2025-07-03 12:57:23,527] A new study created in memory with name: tune_model


In [28]:
# optimize

study.optimize(func=tune_ridge, n_trials=100, n_jobs=-1, show_progress_bar=True)

[I 2025-07-03 12:57:36,050] Trial 11 finished with value: 0.07917175810215299 and parameters: {'alpha': 93.8737352780069}. Best is trial 11 with value: 0.07917175810215299.
[I 2025-07-03 12:57:36,066] Trial 12 finished with value: 0.07919576541246127 and parameters: {'alpha': 76.45352174661457}. Best is trial 11 with value: 0.07917175810215299.
[I 2025-07-03 12:57:36,092] Trial 9 finished with value: 0.07916691057601613 and parameters: {'alpha': 97.69676391439943}. Best is trial 9 with value: 0.07916691057601613.
[I 2025-07-03 12:57:36,093] Trial 3 finished with value: 0.0791821622646131 and parameters: {'alpha': 86.07300273061423}. Best is trial 9 with value: 0.07916691057601613.
[I 2025-07-03 12:57:36,098] Trial 15 finished with value: 0.07919207111341815 and parameters: {'alpha': 79.0092330430374}. Best is trial 9 with value: 0.07916691057601613.
[I 2025-07-03 12:57:36,100] Trial 5 finished with value: 0.07917351821416717 and parameters: {'alpha': 92.52718144616162}. Best is trial 9

[I 2025-07-03 12:57:36,103] Trial 7 finished with value: 0.07917019271839656 and parameters: {'alpha': 95.08974463606624}. Best is trial 0 with value: 0.07916450727398355.
[I 2025-07-03 12:57:36,103] Trial 1 finished with value: 0.07919346481038782 and parameters: {'alpha': 78.03742898918262}. Best is trial 0 with value: 0.07916450727398355.
[I 2025-07-03 12:57:36,105] Trial 13 finished with value: 0.07916614306215745 and parameters: {'alpha': 98.31510036790853}. Best is trial 0 with value: 0.07916450727398355.


Best trial: 0. Best value: 0.0791645:  14%|█▍        | 14/100 [00:04<00:19,  4.35it/s]

[I 2025-07-03 12:57:36,106] Trial 10 finished with value: 0.079252033022699 and parameters: {'alpha': 42.03240545196022}. Best is trial 0 with value: 0.07916450727398355.
[I 2025-07-03 12:57:36,107] Trial 6 finished with value: 0.07917881397844158 and parameters: {'alpha': 88.54685087861529}. Best is trial 0 with value: 0.07916450727398355.
[I 2025-07-03 12:57:36,108] Trial 4 finished with value: 0.07917235929512643 and parameters: {'alpha': 93.41265426910479}. Best is trial 0 with value: 0.07916450727398355.
[I 2025-07-03 12:57:36,109] Trial 8 finished with value: 0.07924779861391086 and parameters: {'alpha': 44.28779795832599}. Best is trial 0 with value: 0.07916450727398355.


Best trial: 0. Best value: 0.0791645:  20%|██        | 20/100 [00:05<00:10,  7.92it/s]

...

[I 2025-07-03 12:57:39,018] Trial 71 finished with value: 0.07922825702228024 and parameters: {'alpha': 55.4536680393182}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:39,041] Trial 72 finished with value: 0.07916421778022212 and parameters: {'alpha': 99.87600031935348}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:39,123] Trial 74 finished with value: 0.07923821684870126 and parameters: {'alpha': 49.62263514946214}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:39,163] Trial 73 finished with value: 0.07916412955180069 and parameters: {'alpha': 99.94787063711851}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:39,167] Trial 75 finished with value: 0.07924622334332734 and parameters: {'alpha': 45.13970127478494}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:39,208] Trial 76 finished with value: 0.0792497965544503 and parameters: {'alpha': 43.220501574352355}. Best is

Best trial: 39. Best value: 0.0791641:  94%|█████████▍| 94/100 [00:08<00:00, 12.49it/s]

[I 2025-07-03 12:57:40,234] Trial 80 finished with value: 0.07916765675312003 and parameters: {'alpha': 97.09933563309269}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:40,239] Trial 93 finished with value: 0.07916800848535487 and parameters: {'alpha': 96.81885061130035}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:40,240] Trial 86 finished with value: 0.07916890344854323 and parameters: {'alpha': 96.10786359808417}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:40,241] Trial 81 finished with value: 0.07916737218345625 and parameters: {'alpha': 97.32655193433266}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:40,242] Trial 83 finished with value: 0.07916754052175311 and parameters: {'alpha': 97.19213350683098}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:40,245] Trial 92 finished with value: 0.07916760048437048 and parameters: {'alpha': 97.14425912229945}. Best i

Best trial: 39. Best value: 0.0791641: 100%|██████████| 100/100 [00:08<00:00, 11.59it/s]

[I 2025-07-03 12:57:40,524] Trial 95 finished with value: 0.07916545928775069 and parameters: {'alpha': 98.86659940405437}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:40,571] Trial 96 finished with value: 0.07916416746289191 and parameters: {'alpha': 99.91698799264327}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:40,579] Trial 97 finished with value: 0.07916435899748578 and parameters: {'alpha': 99.76097426331053}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:40,588] Trial 98 finished with value: 0.07916412240605206 and parameters: {'alpha': 99.95369170078395}. Best is trial 39 with value: 0.07916408773809665.
[I 2025-07-03 12:57:40,593] Trial 99 finished with value: 0.0791650579860793 and parameters: {'alpha': 99.19191553748408}. Best is trial 39 with value: 0.07916408773809665.


In [29]:
# best parameters

study.best_params


{'alpha': 99.98193326743872}

In [30]:
# best value

study.best_value

0.07916408773809665